# 🚀 Model 1. XGBoost + Optuna Tuning

트리 기반 부스팅 모델의 정석인 **XGBoost**를 사용하여 예측 모델을 만듭니다.
단순 학습이 아니라 **Optuna**를 활용하여 최적의 하이퍼파라미터(학습률, 깊이 등)를 자동으로 찾습니다.

### 📋 진행 순서
1.  데이터 로드 (`train_enriched.csv`, `test_enriched.csv`)
2.  Optuna를 이용한 하이퍼파라미터 최적화 (Hyperparameter Tuning)
3.  최적의 파라미터로 최종 모델 학습
4.  결과 예측 및 제출 파일 생성 (`submission_xgb_optuna.csv`)

In [2]:
# ==========================================
# Model 1. XGBoost with Optuna Tuning
# ==========================================
import pandas as pd
import numpy as np
import os
import warnings
import xgboost as xgb
import optuna
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

warnings.filterwarnings('ignore')

# ----------------------------------------------------------
# 1. 데이터 로드 및 준비
# ----------------------------------------------------------
PREPROCESS_DIR = './data_preprocess'
SUBMISSION_DIR = './open_track1'

TRAIN_PATH = os.path.join(PREPROCESS_DIR, 'train_enriched.csv')
TEST_PATH = os.path.join(PREPROCESS_DIR, 'test_enriched.csv')
SUBMISSION_PATH = os.path.join(SUBMISSION_DIR, 'sample_submission.csv')

print("데이터 로딩 중...")
train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)

# 인코딩 (Label Encoding)
cat_features = ['type_name', 'prev_type_name']
le = LabelEncoder()

for col in cat_features:
    all_values = pd.concat([train_df[col].astype(str), test_df[col].astype(str)]).unique()
    le.fit(all_values)
    train_df[col] = le.transform(train_df[col].astype(str))
    test_df[col] = le.transform(test_df[col].astype(str))

# 피처 선택
feature_cols = [
    'start_x', 'start_y', 'type_name', 'team_id', 'time_seconds',
    'prev_type_name', 'prev_end_x', 'prev_end_y',
    'dist_to_goal', 'angle_to_goal', 'dist_to_center'
]
target_cols = ['end_x', 'end_y']

X = train_df[feature_cols]
y = train_df[target_cols]
X_test = test_df[feature_cols]
test_ids = test_df['game_episode']

# 튜닝용 데이터 분리 (8:2)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"학습 데이터 준비 완료: {X_train.shape}")

# ----------------------------------------------------------
# 2. Optuna를 이용한 하이퍼파라미터 튜닝
# ----------------------------------------------------------
def objective(trial):
    """Optuna가 최적화할 목표 함수 (RMSE 최소화)"""
    
    # 튜닝할 파라미터 범위 설정
    params = {
        'objective': 'reg:squarederror',
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 7),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 10),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 10),
        'random_state': 42,
        'n_jobs': -1
    }
    
    # XGBoost는 다중 타겟(Multi-output)을 기본 지원하지 않으므로,
    # MultiOutputRegressor를 쓰거나 각각 학습해야 하지만,
    # 여기서는 편의상 X축과 Y축에 대해 동일한 하이퍼파라미터로 평가합니다.
    # (실제로는 각각 튜닝하면 더 좋지만 시간이 2배 걸림)
    
    # X축 예측 모델
    model_x = xgb.XGBRegressor(**params)
    model_x.fit(X_train, y_train['end_x'])
    pred_x = model_x.predict(X_val)
    
    # Y축 예측 모델
    model_y = xgb.XGBRegressor(**params)
    model_y.fit(X_train, y_train['end_y'])
    pred_y = model_y.predict(X_val)
    
    # 평균 RMSE 계산
    rmse_x = np.sqrt(mean_squared_error(y_val['end_x'], pred_x))
    rmse_y = np.sqrt(mean_squared_error(y_val['end_y'], pred_y))
    
    return (rmse_x + rmse_y) / 2

print("\nOptuna 튜닝 시작 (20회 시도)...")
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20) # 시간 관계상 20회만 (더 늘려도 됨)

print(f"\nBest Trial: {study.best_trial.value:.4f}")
print("Best Params:", study.best_params)

# ----------------------------------------------------------
# 3. 최적 파라미터로 최종 학습 및 예측
# ----------------------------------------------------------
print("\n최적 파라미터로 전체 데이터 재학습 중...")

best_params = study.best_params
best_params['random_state'] = 42
best_params['n_jobs'] = -1

# 전체 데이터로 학습 (X, y 전체 사용)
final_model_x = xgb.XGBRegressor(**best_params)
final_model_y = xgb.XGBRegressor(**best_params)

final_model_x.fit(X, y['end_x'])
final_model_y.fit(X, y['end_y'])

# 예측
pred_x = final_model_x.predict(X_test)
pred_y = final_model_y.predict(X_test)

# Clipping (경기장 규격 보정)
pred_x = np.clip(pred_x, 0, 105)
pred_y = np.clip(pred_y, 0, 68)

# ----------------------------------------------------------
# 4. 제출 파일 생성
# ----------------------------------------------------------
submission_df = pd.DataFrame({
    'game_episode': test_ids,
    'end_x': pred_x,
    'end_y': pred_y
})

sample_submission = pd.read_csv(SUBMISSION_PATH)
final_submission = pd.merge(sample_submission[['game_episode']], submission_df, on='game_episode', how='left')
final_submission.fillna(50.0, inplace=True)

save_path = 'submission_xgb_optuna.csv'
final_submission.to_csv(save_path, index=False)

print(f"\nXGBoost 튜닝 완료! 결과 저장됨: {save_path}")

[I 2025-12-02 14:02:59,456] A new study created in memory with name: no-name-a98f9736-abc8-4011-a36c-563ddc186c8f


데이터 로딩 중...
학습 데이터 준비 완료: (12348, 11)

Optuna 튜닝 시작 (20회 시도)...


[I 2025-12-02 14:03:01,818] Trial 0 finished with value: 14.482791937255477 and parameters: {'n_estimators': 179, 'learning_rate': 0.16947298447891884, 'max_depth': 8, 'min_child_weight': 4, 'subsample': 0.7178046042891915, 'colsample_bytree': 0.7909285413492447, 'reg_alpha': 9.202737634194198, 'reg_lambda': 3.2333572562267765}. Best is trial 0 with value: 14.482791937255477.
[I 2025-12-02 14:03:04,705] Trial 1 finished with value: 14.247589721708668 and parameters: {'n_estimators': 701, 'learning_rate': 0.10348342727249248, 'max_depth': 6, 'min_child_weight': 3, 'subsample': 0.9104459438814834, 'colsample_bytree': 0.9219632855112035, 'reg_alpha': 4.287898428769275, 'reg_lambda': 6.295557820568888}. Best is trial 1 with value: 14.247589721708668.
[I 2025-12-02 14:03:06,874] Trial 2 finished with value: 14.521789612066586 and parameters: {'n_estimators': 423, 'learning_rate': 0.11172770383780894, 'max_depth': 7, 'min_child_weight': 7, 'subsample': 0.7130841582114915, 'colsample_bytree':


Best Trial: 13.5326
Best Params: {'n_estimators': 849, 'learning_rate': 0.013625412480311919, 'max_depth': 5, 'min_child_weight': 2, 'subsample': 0.8804309946238006, 'colsample_bytree': 0.7937930843849573, 'reg_alpha': 1.6942488450715314, 'reg_lambda': 2.997386705805371}

최적 파라미터로 전체 데이터 재학습 중...

XGBoost 튜닝 완료! 결과 저장됨: submission_xgb_optuna.csv
